In [1]:
#jaccard distance --> bad because ignore ratings
#cosine distance --> treat missing values as negative
#pearson distance/centred cosin distance --> good

In [1]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial.distance import correlation

In [2]:
data=pd.read_csv("/home/angps/Documents/DSC3216 (1)/project/the-movies-dataset/ratings_small.csv")
data.head(2)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179


In [3]:
ratings = data.pivot('userId','movieId','rating').fillna(0)
ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
'''
u = data.groupby('userId')['rating'].count()
user = u.sort_values(ascending = False)

m = data.groupby('movieId')['rating'].count()
movie = m.sort_values(ascending = False)

joined = data.join(user, on='userId', how = 'inner', rsuffix='_r')
joined = joined.join(movie, on='movieId', how = 'inner', rsuffix = '_r')

ratings=pd.crosstab(joined.userId, joined.movieId, joined.rating, aggfunc=np.sum)

ratings = ratings.fillna(value=0)
ratings.head()
'''

"\nu = data.groupby('userId')['rating'].count()\nuser = u.sort_values(ascending = False)\n\nm = data.groupby('movieId')['rating'].count()\nmovie = m.sort_values(ascending = False)\n\njoined = data.join(user, on='userId', how = 'inner', rsuffix='_r')\njoined = joined.join(movie, on='movieId', how = 'inner', rsuffix = '_r')\n\nratings=pd.crosstab(joined.userId, joined.movieId, joined.rating, aggfunc=np.sum)\n\nratings = ratings.fillna(value=0)\nratings.head()\n"

In [4]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(ratings, test_size=0.20,random_state=0)

In [5]:
test.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
665,0.0,3.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
311,3.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,2.0,2.0,0.0,0.0,4.5,4.0,0.0,0.0,0.0,3.0,...,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
623,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,4.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
520,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
232,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
def distance_train(user1,user2):
        try:
            user1Ratings = train.transpose()[user1]
            user2Ratings = train.transpose()[user2]
            distance = correlation(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

#for test
def distance(testuser,trainuser):
        try:
            user1Ratings = test.transpose()[testuser]
            user2Ratings = train.transpose()[trainuser]
            distance = correlation(user1Ratings,user2Ratings)
        except: 
            distance = np.NaN
        return distance

def get_k_nearest(new_userid,k):
    table = pd.DataFrame(train.index)
    table["dist"] = table["userId"].apply(lambda x: distance(new_userid,x))   #calc distance of each user to the new user
    K_nearest_users = table.sort_values(["dist"],ascending=True)[:k]  #get lowest k distance (highest similarity)
    K_nearest_users.set_index("userId",inplace=True)
    return K_nearest_users

def ratings_of_k_nearest(new_userid,k):
    k_nearest = get_k_nearest(new_userid,k)
    rating = train[train.index.isin(k_nearest.index)]  # index is the user id. so get the movie ratings of all users in the k nearest users
    return rating

def get_ratings(train_user_id,movieid):
    return train.at[train_user_id,movieid]
def pred_ratings(new_userid,k):  #using weighted average
    '''Input new user ID and k, the number of neighours selected'''
    if k<20:
        return 'Please choose a higher k'
    else:
        k_nearest = get_k_nearest(new_userid,k)   #get k nearest users
        rating = ratings_of_k_nearest(new_userid,k)   #get ratings of the k nearest users
        merged = k_nearest.merge(rating,left_index=True,right_index=True) #combine to find the ratings of each movie for each k nearest users    
        pred = pd.DataFrame(rating.columns)
        pred.set_index("movieId",inplace=True)
        pred["ratings"] = 0.0
    for i in range(1,len(merged.columns)):
        col = merged.columns[i]
        valid_col = [t for t in merged.index if merged.loc[t, col]!=0]
        if len(valid_col)<5:   #can adjust how many valid ratings
            pred.at[col,'ratings'] = 0
        else:
            temp = merged.loc[valid_col,['dist',col]]
            temp['product'] = temp.prod(1)
            pred.at[col,'ratings'] = temp['product'].sum()/temp['dist'].sum()
    pred = pred.sort_values(["ratings"],ascending=False)
    return pred

def recommend_top_n(new_userid,k,top_n):
    if top_n == 0:
        return pred_ratings(new_userid,k).set_index("movieId")
    else:
        return pred_ratings(new_userid,k)[:top_n].set_index("movieId")
            

In [14]:
confusion_matrix = pd.DataFrame(0,['Like', 'Dislike'],['Predicted Like','Predicted Dislike'])
real = pd.DataFrame(test.loc[665])
#print (real.head(3))
predicted = pd.DataFrame(pred_ratings(665, 20))
#print (predicted.head(3))
combine = real.merge(predicted,left_index=True,right_index=True)
combine = combine[(combine[665]!=0) & (combine['ratings']!=0)]
combine.columns = ['Real','Predicted']
combine['Errors'] = abs(combine['Real']-combine['Predicted'])

real_mean = combine['Real'].mean()
predicted_mean = combine['Predicted'].mean()

print ('Real', real_mean)
print('Predicted', predicted_mean)

def accuracy(df):
    if df['Predicted'] > predicted_mean:
        if df['Real'] > real_mean:
            return 'True Positive'
        else:
            return 'False Positive'
    else:
        if df['Real'] < real_mean:
            return 'True Negative'
        else:
            return 'False Negative'
    
combine['Accuracy'] = combine.apply(accuracy,1)
print (combine.head(10))      
combine['Accuracy'].value_counts()['False Negative']

confusion_matrix.loc['Like','Predicted Like'] += combine['Accuracy'].value_counts()['True Positive']
confusion_matrix.loc['Like','Predicted Dislike'] += combine['Accuracy'].value_counts()['False Negative']
confusion_matrix.loc['Dislike','Predicted Like'] += combine['Accuracy'].value_counts()['False Positive']
confusion_matrix.loc['Dislike','Predicted Dislike'] += combine['Accuracy'].value_counts()['True Negative']
print (confusion_matrix)

Real 3.4681818181818183
Predicted 3.5238330037810552
         Real  Predicted    Errors        Accuracy
movieId                                           
2         3.0   3.406773  0.406773   True Negative
5         3.0   2.930000  0.070000   True Negative
16        4.0   3.345503  0.654497  False Negative
32        4.0   3.665430  0.334570   True Positive
34        2.0   4.048329  2.048329  False Positive
39        2.0   3.532598  1.532598  False Positive
48        4.0   2.645480  1.354520  False Negative
62        3.0   3.806660  0.806660  False Positive
104       4.0   3.314833  0.685167  False Negative
165       3.0   3.293311  0.293311   True Negative
         Predicted Like  Predicted Dislike
Like                 67                 39
Dislike              50                 64


In [15]:

    
confusion_matrix = pd.DataFrame(0,['Like', 'Dislike'],['Predicted Like','Predicted Dislike'])

def cm(df):
    def accuracy(df):
        if df['Predicted'] > predicted_mean:
            if df['Real'] > real_mean:
                return 'True Positive'
            else:
                return 'False Positive'
        else:
            if df['Real'] < real_mean:
                return 'True Negative'
            else:
                return 'False Negative'
    for user in test.index:
        real = pd.DataFrame(test.loc[user])
        predicted = pd.DataFrame(pred_ratings(user,25))
        combine = real.merge(predicted,left_index=True,right_index=True)
        combine = combine[(combine[user]!=0) & (combine['ratings']!=0)]
        combine.columns = ['Real','Predicted']
        real_mean = combine['Real'].mean()
        predicted_mean = combine['Predicted'].mean()
        combine['Accuracy'] = combine.apply(accuracy,1)
        if 'True Positive' in combine['Accuracy'].value_counts():
            df.loc['Like','Predicted Like'] += combine['Accuracy'].value_counts()['True Positive']
        if 'False Negative' in combine['Accuracy'].value_counts():
            df.loc['Like','Predicted Dislike'] += combine['Accuracy'].value_counts()['False Negative']
        if 'False Positive' in combine['Accuracy'].value_counts():
            df.loc['Dislike','Predicted Like'] += combine['Accuracy'].value_counts()['False Positive']
        if 'True Negative' in combine['Accuracy'].value_counts():
            df.loc['Dislike','Predicted Dislike'] += combine['Accuracy'].value_counts()['True Negative']

cm(confusion_matrix)

print (confusion_matrix)

         Predicted Like  Predicted Dislike
Like               4292               2108
Dislike            2199               3366


In [16]:
print (confusion_matrix)
print ("Accuracy: " + str((confusion_matrix.loc["Like","Predicted Like"]+confusion_matrix.loc["Dislike","Predicted Dislike"])/(confusion_matrix.loc["Like","Predicted Like"]+confusion_matrix.loc["Dislike","Predicted Dislike"]+confusion_matrix.loc["Like","Predicted Dislike"]+confusion_matrix.loc["Dislike","Predicted Like"])))
print ("True Positive " + str(confusion_matrix.loc["Like","Predicted Like"]/(confusion_matrix.loc["Like","Predicted Like"]+confusion_matrix.loc["Like","Predicted Dislike"])))
print ("True Negative " + str(confusion_matrix.loc["Dislike","Predicted Dislike"]/(confusion_matrix.loc["Dislike","Predicted Dislike"]+confusion_matrix.loc["Dislike","Predicted Like"])))
print ("False Positive " + str(confusion_matrix.loc["Dislike","Predicted Like"]/(confusion_matrix.loc["Dislike","Predicted Like"]+confusion_matrix.loc["Dislike","Predicted Dislike"])))
print ("False Negative " + str(confusion_matrix.loc["Like","Predicted Dislike"]/(confusion_matrix.loc["Like","Predicted Dislike"]+confusion_matrix.loc["Like","Predicted Like"])))
       

         Predicted Like  Predicted Dislike
Like               4292               2108
Dislike            2199               3366
Accuracy: 0.64003343084
True Positive 0.670625
True Negative 0.604851752022
False Positive 0.395148247978
False Negative 0.329375


In [17]:
print (pred_ratings(77,20).head())

print (test.loc[77,pred_ratings(77,20).head().index])

x=pd.DataFrame(test.loc[77])
x=x[x[77]!=0]
print ('User average rating: ', x.mean())

          ratings
movieId          
1136     4.631477
1196     4.563273
2571     4.500777
2291     4.497434
2959     4.496521
movieId
1136    4.5
1196    4.5
2571    4.0
2291    3.5
2959    4.0
Name: 77, dtype: float64
User average rating:  77    3.319048
dtype: float64


In [ ]:
def pred_ratings(new_userid,k):   #using weighted average
    '''Input new user ID and k, the number of neighours selected'''
    if k<20:
        return 'Please choose a higher k'
    else:
        k_nearest = get_k_nearest(new_userid,k)   #get k nearest users
        rating = ratings_of_k_nearest(new_userid,k)   #get ratings of the k nearest users
        merged = k_nearest.merge(rating,left_index=True,right_index=True) #combine to find the ratings of each movie for each k nearest users    
        pred = pd.DataFrame(rating.columns)
        pred.set_index("movieId",inplace=True)
        pred["ratings"] = 0.0
        for i in range(1,len(merged.columns)):
            col = merged.columns[i]
            valid_col = [t for t in merged.index if merged.loc[t, col]!=0]
            if len(valid_col)<5:   #can adjust how many valid ratings
                pred.at[col,'ratings'] = 0
            else:
                temp = merged.loc[valid_col,['dist',col]]
                temp['product'] = temp.prod(1)
                pred.at[col,'ratings'] = temp['product'].sum()/temp['dist'].sum()
        pred = pred.sort_values(["ratings"],ascending=False)
        #remove = data[data['userId']==new_userid]['movieId'].values
        #pred.drop(remove,inplace=True)
        return pred